In [11]:
#r ".\binaries\\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Creation of Differential Operator


Velocity field and Exact solution for Dirichlet Boundary

Regarding the spatial part of the operator, we re-use the SIP-implementation 
which is already available in the `BoSSS.Solution.NSECommon` library.
For this implementation, whe have to specify the diffusion coefficient (here to -0.5)
as well as the Location of the Dirichlet boundary.

In [12]:
using BoSSS.Foundation.XDG;
class Laplace : BoSSS.Solution.NSECommon.SIPLaplace
{
    
    public Laplace(): base(1.4, "u"){
    }

    protected override bool IsDirichlet(ref CommonParamsBnd inp) {
        return true;
    }
    
    // diffusion coefficient
    override public double Nu(double[] x, double[] p, int jCell) {
        return -1.0;
    }
    override protected double g_Diri(ref CommonParamsBnd inp) { 
        return 0.0;
    }
}
class XLaplace:Laplace, ISpeciesFilter
{
    string ValidSpecies;

    string ISpeciesFilter.ValidSpecies => this.ValidSpecies;

    public XLaplace(string ValidSpecies): base(){
        this.ValidSpecies=ValidSpecies;        
    } 
}


In [20]:
class InterfaceFlux : ILevelSetForm {
    public IEdgeForm baseflux;
    public InterfaceFlux(IEdgeForm baseflux) {
        this.baseflux = baseflux;
    }
    public int LevelSetIndex => 0;

    public string PositiveSpecies => "A";

    public string NegativeSpecies => "B";

    public TermActivationFlags LevelSetTerms => TermActivationFlags.AllOn;

    public IList<string> ArgumentOrdering {
        get { return new string[] { "u" }; }
    }

    public IList<string> ParameterOrdering => null;

    public double InnerEdgeForm(ref CommonParams inp, double[] _uIN, double[] _uOUT, double[,] _Grad_uIN, double[,] _Grad_uOUT, double _vIN, double _vOUT, double[] _Grad_vIN, double[] _Grad_vOUT) {
        var uOUT = new double[] { 0 };
        return baseflux.InnerEdgeForm(ref inp, _uIN, uOUT, _Grad_uIN, _Grad_uOUT, _vIN, _vOUT, _Grad_vIN, _Grad_vOUT);
    }
}

#### Operator assembly
We compose the differential operator from the previously created 
`Laplace` implementation and add the `ScalarTransportFlux`. 

In [22]:
var Op = new XDifferentialOperatorMk2(1, 0, 1, QuadOrderFunc.Linear(),new string[] {"A","B"}, "u", "R1");

Op.EquationComponents["R1"].Add(new XLaplace("A")); // adding Laplace
Op.EquationComponents["R1"].Add(new InterfaceFlux(new Laplace()));
Op.Commit();

# <span style="color:red"> test if cell length scales are there </span>

In [24]:
using ilPSP.LinSolvers;
var nC = 2;
var nodes = GenericBlas.Linspace(0.0, 2.0, nC + 1);
var grid = Grid2D.Cartesian2DGrid(nodes, nodes);
var levelSet = new LevelSet(new Basis(grid, 2), "levelSet");
//levelSet.ProjectField((double[] X) => MyGlobals.levelSet(X, 0.0));
levelSet.ProjectField((double[] X) => X[0]-0.5);
var LsTrk = new LevelSetTracker(grid.GridData, XQuadFactoryHelper.MomentFittingVariants.Saye, 1, new string[] {  "A","B" }, levelSet);
//LsTrk.UpdateTracker(0.0);
var xdgbasis = new XDGBasis(LsTrk, 2);
var u = new XDGField(xdgbasis, "u");
var U = new XDGField[1]; U[0] = u;
var coef_prov = Op.OperatorCoefficientsProvider;
int trackerHistoryIndex=0;
double time=0.0;
var result = coef_prov(LsTrk, LsTrk.SpeciesIdS[0], 4, trackerHistoryIndex,  time);
var U_map = new CoordinateMapping(U);
double[] res = new double[U_map.TotalLength];
var mtxBuilder = Op.GetMatrixBuilder(LsTrk, U_map, null, U_map);
mtxBuilder.time = 0.0;


var Op_Agglomeration = LsTrk.GetAgglomerator(LsTrk.SpeciesIdS.ToArray(), Op.GetOrderFromQuadOrderFunction(U_map.BasisS, null, U_map.BasisS),Op.AgglomerationThreshold);
var AgglomeratedCellLengthScales = Op_Agglomeration.CellLengthScales;
foreach(var kv in AgglomeratedCellLengthScales) {
    mtxBuilder.CellLengthScales[kv.Key] = kv.Value;               
}
BlockMsrMatrix OpMtx = new BlockMsrMatrix(U_map);
//double[] OpAffine=new double[U_map.TotalLength];
mtxBuilder.ComputeMatrix(OpMtx, res);
res

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Solution.NSECommon.SIPLaplace.GetPenalty(Int32 jCellIn, Int32 jCellOut) in C:\experimental\public\src\L3-solution\BoSSS.Solution.NSECommon\ipLaplace.cs:line 132
   at BoSSS.Solution.NSECommon.SIPLaplace.InnerEdgeForm(CommonParams& inp, Double[] _uA, Double[] _uB, Double[,] _Grad_uA, Double[,] _Grad_uB, Double _vA, Double _vB, Double[] _Grad_vA, Double[] _Grad_vB) in C:\experimental\public\src\L3-solution\BoSSS.Solution.NSECommon\ipLaplace.cs:line 197
   at Submission#21.InterfaceFlux.InnerEdgeForm(CommonParams& inp, Double[] _uIN, Double[] _uOUT, Double[,] _Grad_uIN, Double[,] _Grad_uOUT, Double _vIN, Double _vOUT, Double[] _Grad_vIN, Double[] _Grad_vOUT)
   at BoSSS.Foundation.XDG.LinearLevelSetFormVectorizer.GetCoeff(Double& d1, Double& d2, CommonParams& inp, Double[] uA, Double[] uB, Double[,] Grad_uA, Double[,] Grad_uB, Double& vA, Double& vB, Double[] Grad_vA, Double[] Grad_